In [1]:
import sys; sys.path.append('/rds/general/user/cb2714/home/'); from wwa import *
cproj = cartopy.crs.PlateCarree()

xn, xx, yn, yx = [-85,-75,6,10]

---
## **Compile CHIRPS**

In [2]:
ds = xr.open_mfdataset("../00_WWA_project_folder/live/data/chirps_05/*.nc").sel(longitude = slice(xn,xx), latitude = slice(yn, yx))
ds.to_netcdf("data/chirps05_panama.nc")

## **Compile MSWEP**

In [3]:
# cut out yearly slices
for y in range(2023, 1978, -1):
    
    new_fnm = "mswep/mswep_panama_"+str(y)+".nc"
    if os.path.exists(new_fnm): continue
    
    print(y)
    da = xr.open_mfdataset("../00_WWA_project_folder/live/data/mswep_*/"+str(y)+"*.nc").sel(lon = slice(xn,xx), lat = slice(yx, yn))
    wrap_lon(da).rename(precipitation = "pr").to_netcdf(new_fnm)

2023


In [4]:
# compile into single file
xr.open_mfdataset("mswep/mswep_panama_*.nc").to_netcdf("data/mswep_panama.nc")

## **MSWX tmax & tmin**

In [7]:
varnm = "tmin"

# cut out yearly slices
for y in range(2023, 1978, -1):
    
    new_fnm = "mswep/mswx_"+varnm+"_panama_"+str(y)+".nc"
    if os.path.exists(new_fnm): continue
    
    print(y)
    da = xr.open_mfdataset("../00_WWA_project_folder/live/data/mswx/"+varnm+"_*/"+str(y)+"*.nc").sel(lon = slice(xn,xx), lat = slice(yx, yn))
    wrap_lon(da).rename(air_temperature = varnm).to_netcdf(new_fnm)

2023


In [8]:
# compile into single file
xr.open_mfdataset("mswep/mswx_"+varnm+"_panama_*.nc").to_netcdf("data/mswx_"+varnm+"_panama.nc")

---
## **ERA5land**

### **Download**

In [ ]:
import os
import cdsapi

c = cdsapi.Client()

for y in [str(y) for y in range(2023,1950,-1)]:
#     for m in [str(m).rjust(2, "0") for m in range(13)[1:]]:
        
        new_fnm = 'era5-land/era5land_panama_tp_'+y+'.nc'
        if os.path.exists(new_fnm): continue
        print(new_fnm)

        c.retrieve(
            'reanalysis-era5-land',
            {
                'variable': [
                    'total_precipitation',
                ],
                'year': y,
                'month': [
                    '01', '02', '03',
                    '04', '05', '06',
                    '07', '08', '09',
                    '10', '11', '12'],
                'day': [
                    '01', '02', '03',
                    '04', '05', '06',
                    '07', '08', '09',
                    '10', '11', '12',
                    '13', '14', '15',
                    '16', '17', '18',
                    '19', '20', '21',
                    '22', '23', '24',
                    '25', '26', '27',
                    '28', '29', '30', '31',
                ],
                'time': [
                    '00:00', '01:00', '02:00',
                    '03:00', '04:00', '05:00',
                    '06:00', '07:00', '08:00',
                    '09:00', '10:00', '11:00',
                    '12:00', '13:00', '14:00',
                    '15:00', '16:00', '17:00',
                    '18:00', '19:00', '20:00',
                    '21:00', '22:00', '23:00',
                ],
                'area': [
                    9.7, -80.25, 8.7,
                    -79.25,
                ],
                'format': 'netcdf',
            },
            new_fnm)
        

### **Compile**

In [8]:
ds = xr.open_mfdataset([fnm for fnm in glob.glob("era5-land/*.nc")]).max("expver").tp
ds = ds.resample(time = "D").sum()
ds.to_netcdf("data/era5land_panama.nc")